In [ ]:
import yfinance as yf
import csv
import json
# 語法是 yf.download(股票代號, start=開始日期, end=完結日期)
data = yf.download("2330.TW", start = '2023-01-01')
data.to_csv('tsmc.csv')

def make_json(csvFilePath, jsonFilePath):
    data_dict = {}
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        for rows in csvReader:
             
            key = rows['Date']
            data_dict[key] = rows
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        # 寫入資料：json.dump(資料, 檔案物件)
        jsonf.write(json.dumps(data_dict, indent=4))

csvFilePath = r'tsmc.csv'
jsonFilePath = r'tsmc.json'
 
make_json(csvFilePath, jsonFilePath)

In [ ]:
import sqlite3
import json

def create_table(conn):
    # 呼叫 conn.cursor() 建立 Cursor 物件
    cursor = conn.cursor()
    # 呼叫 cursor.execute() 執行 CRUD 操作 ， 新增、修改、刪除、查詢 ( CRUD )
    cursor.execute(
        '''
        CREATE TABLE IF NOT EXISTS 台積電 (
            "id"    INTEGER NOT NULL,
            "日期"	INTEGER NOT NULL,
            "開市"	INTEGER NOT NULL,
            "最高"	INTEGER NOT NULL,
            "最低"	INTEGER NOT NULL,
            "收市"	INTEGER NOT NULL,
            "經調整收市價"	INTEGER NOT NULL,
            "成交量"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT)
        );
        '''
    )
    conn.commit()
def insert_data(conn,values):
    cursor = conn.cursor()
    sql = '''
        INSERT INTO 台積電 (日期, 開市, 最高, 最低, 收市, 經調整收市價, 成交量)
        VALUES(?,?,?,?,?,?,?)
        '''
    # 傳回 Cursor 物件
    cursor.execute(sql, values)
    # 寫回資料庫
    conn.commit()
# 連接資料庫檔案
conn = sqlite3.connect("stock.db")
create_table(conn)
# 寫回資料庫
conn.commit()

with open("tsmc.json",mode="r") as file:
    data = json.load(file)
    for item in data:
        insert_data(conn,(item[0], item[1], item[2], item[3], item[4], item[5], item[6]))